In [ ]:
!pip install --upgrade scikit-learn
!pip install torch
!pip install imblearn
!pip install catboost
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
      Successfully uninstalled scikit-learn-1.6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.9 MB/s eta 0:00:00


In [ ]:
#Import All Libraies
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load datasets
train = pd.read_csv("/content/Train.csv")
test = pd.read_csv("/content/Test.csv")

In [ ]:
# Feature Engineering
def feature_engineer(df):
    df['days_to_repay'] = (pd.to_datetime(df['due_date']) - pd.to_datetime(df['disbursement_date'])).dt.days
    df['repay_ratio'] = df['Total_Amount_to_Repay'] / df['Total_Amount']
    df['amount_duration_ratio'] = df['Total_Amount'] / df['duration']
    df['disbursement_month'] = pd.to_datetime(df['disbursement_date']).dt.month
    df['amount_duration_interaction'] = df['Total_Amount'] * df['duration']
    df['repay_duration_interaction'] = df['Total_Amount_to_Repay'] * df['duration']
    return df

In [ ]:
# Apply feature engineering
train = feature_engineer(train)
test = feature_engineer(test)

In [ ]:
# Encode categorical features
categorical_features = ['loan_type', 'New_versus_Repeat', 'country_id']
for feature in categorical_features:
    unique_labels = train[feature].unique()
    label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
    train[feature] = train[feature].map(label_mapping)
    if feature in test.columns:
        test[feature] = test[feature].map(label_mapping)
        test[feature] = test[feature].fillna(-1)

In [ ]:
# Drop unnecessary columns
drop_columns = ['ID', 'customer_id', 'tbl_loan_id', 'lender_id', 'disbursement_date', 'due_date']
X = train.drop(columns=drop_columns + ['target']).values
y = train['target'].values
X_test = test.drop(columns=drop_columns).values

In [ ]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [ ]:
# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Unsqueeze for binary classification
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)  # Unsqueeze for binary classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [ ]:
# Create Dataset and DataLoader
class TabularDataset(Dataset):
    def _init_(self, X, y):
        self.X = X
        self.y = y

    def _len_(self):
        return len(self.X)

    def _getitem_(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# Define the Deep Learning Model
class NeuralNetwork(nn.Module):
    def _init_(self, input_size, hidden_size=128, dropout_rate=0.3):
        super(NeuralNetwork, self)._init_()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Create Dataset and DataLoader
class TabularDataset(Dataset):
    def __init__(self, X, y): # Changed _init_ to __init__
        self.X = X
        self.y = y

    def __len__(self): # Changed _len_ to __len__
        return len(self.X)

    def __getitem__(self, idx): # Changed _getitem_ to __getitem__
        return self.X[idx], self.y[idx]

In [ ]:
# Define the Deep Learning Model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size=128, dropout_rate=0.3): # Changed _init_ to __init__
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Hyperparameter Tuning for Neural Network
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [32, 64, 128],
    'hidden_size': [64, 128, 256],
    'dropout': [0.2, 0.3, 0.4]
}

best_val_f1 = 0
best_params = { # Initialize best_params with default values
    'learning_rate': 0.001,
    'batch_size': 32,
    'hidden_size': 128,
    'dropout_rate': 0.3
} # If no better parameters are found, these will be used.

# ... (rest of your hyperparameter tuning loop) ...

In [ ]:
# Assuming 'best_params' is defined from your hyperparameter tuning
input_size = X_train.shape[1]
model = NeuralNetwork(input_size, hidden_size=best_params['hidden_size'], dropout_rate=best_params['dropout_rate']) # Instantiate the model
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'])

train_dataset = TabularDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
epochs = 30  # Train for more epochs with the optimal hyperparameters
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}")

# Now you can predict on test data:
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor).squeeze()
    y_test_pred_binary = (y_test_pred >= 0.5).int().numpy()  # Convert to NumPy array for submission

Epoch 1/30, Train Loss: 391.6519
Epoch 2/30, Train Loss: 272.8129
Epoch 3/30, Train Loss: 243.6655
Epoch 4/30, Train Loss: 223.9760
Epoch 5/30, Train Loss: 209.5504
Epoch 6/30, Train Loss: 209.6576
Epoch 7/30, Train Loss: 196.4218
Epoch 8/30, Train Loss: 190.3457
Epoch 9/30, Train Loss: 189.1491
Epoch 10/30, Train Loss: 183.4062
Epoch 11/30, Train Loss: 182.2052
Epoch 12/30, Train Loss: 178.3649
Epoch 13/30, Train Loss: 176.8630
Epoch 14/30, Train Loss: 174.4975
Epoch 15/30, Train Loss: 174.4604
Epoch 16/30, Train Loss: 175.3951
Epoch 17/30, Train Loss: 171.9294
Epoch 18/30, Train Loss: 167.3223
Epoch 19/30, Train Loss: 175.5854
Epoch 20/30, Train Loss: 168.1309
Epoch 21/30, Train Loss: 168.5030
Epoch 22/30, Train Loss: 165.6304
Epoch 23/30, Train Loss: 176.7736
Epoch 24/30, Train Loss: 162.9377
Epoch 25/30, Train Loss: 165.8060
Epoch 26/30, Train Loss: 162.2015
Epoch 27/30, Train Loss: 167.1733
Epoch 28/30, Train Loss: 159.6173
Epoch 29/30, Train Loss: 159.4368
Epoch 30/30, Train Loss

In [ ]:
# Save Predictions to Submission File
submission = pd.DataFrame({
    'ID': test['ID'],
    'target': y_test_pred_binary
})
submission.to_csv("submission_tuned_nn.csv", index=False)
print("Submission file created: submission_tuned_nn.csv")

Submission file created: submission_tuned_nn.csv
